In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/2022-ai-tp-19011845/sample.csv
/kaggle/input/2022-ai-tp-19011845/train.csv
/kaggle/input/2022-ai-tp-19011845/test.csv


# Attack1 코드와의 차이점
* 월, 일열 삭
* BatchNorm1d 함수 사용X
* 레이어 개수 8개로 늘림

# 1) 랜덤시드 고정

In [2]:
import torch
import torch.optim as optim
import torchvision.transforms as transforms
import random

device = 'cuda' if torch.cuda.is_available() else 'cpu'


random.seed(42)
torch.manual_seed(42)
if device == 'cuda':
    torch.cuda.manual_seed_all(42)
    torch.cuda.manual_seed_all(42)
    print("cuda")
    
torch.backends.cudnn.benchmark = False
torch.backends.cudnn.deterministic = True

cuda


# 2) 데이터 셋 불러오기

In [3]:
test = pd.DataFrame(pd.read_csv('/kaggle/input/2022-ai-tp-19011845/test.csv',engine='python',encoding='CP949'))
train = pd.DataFrame(pd.read_csv('/kaggle/input/2022-ai-tp-19011845/train.csv',engine='python',encoding='CP949'))
sample = pd.DataFrame(pd.read_csv('/kaggle/input/2022-ai-tp-19011845/sample.csv',engine='python',encoding='CP949'))

In [4]:
train

,업무일자,역명,승하차,사용자,시각,평균기온(℃),최고기온(℃),최저기온(℃),강수량(mm),평균습도(%rh),탑승인원
0,2019-03-07,목동,승차,어린이,8시,7.0,12.7,3.1,NaN,51.1,0
1,2019-11-20,올림픽공원(한국체대),하차,중고생,21시,0.5,6.3,-4.6,NaN,35.5,0
2,2019-03-09,목동,승차,어린이,12시,7.2,14.9,-0.4,NaN,31.8,19
3,2019-07-07,상일동,하차,일반,8시,26.7,32.5,22.7,NaN,43.3,97
4,2019-05-17,여의나루,승차,우대권,16시,23.6,28.3,17.6,NaN,44.9,99
...,...,...,...,...,...,...,...,...,...,...,...
99995,2019-11-16,마포,하차,외국인,13시,6.8,12.7,3.8,NaN,81.3,2
99996,2019-08-21,김포공항,하차,일반,00시,27.0,30.4,24.4,1.3,67.3,14
99997,2019-02-23,행당,승차,청소년,18시,6.8,14.5,0.5,NaN,41.0,19
99998,2019-05-14,여의나루,하차,중고생,12시,19.6,26.9,12.9,NaN,44.5,0


In [5]:
sample

,index,predict
0,1,0
1,2,0
2,3,0
3,4,0
4,5,0
...,...,...
9995,9996,0
9996,9997,0
9997,9998,0
9998,9999,0


# 3) 데이터전처리
* train 데이터 x, y 분리
* 업무일자열 삭제
* 시각 데이터를 문자열에서 int형으로 전환
* 범주형 데이터 처리
* 데이터 정규화

In [6]:
# train 데이터 x, y 분리
train_y = train['탑승인원']
train_x = train.drop('탑승인원', axis = 1)

In [7]:
# train 데이터 업무일자 월, 일로 나누기
date_list = list(train_x['업무일자'])
month = []
date = []

for day in date_list:
    month.append(int(day[5:7]))
    date.append(int(day[8:10]))
    
train_x['월'] = month
train_x['일'] = date

train_x = train_x.drop('업무일자',axis = 1)
train_x

,역명,승하차,사용자,시각,평균기온(℃),최고기온(℃),최저기온(℃),강수량(mm),평균습도(%rh),월,일
0,목동,승차,어린이,8시,7.0,12.7,3.1,NaN,51.1,3,7
1,올림픽공원(한국체대),하차,중고생,21시,0.5,6.3,-4.6,NaN,35.5,11,20
2,목동,승차,어린이,12시,7.2,14.9,-0.4,NaN,31.8,3,9
3,상일동,하차,일반,8시,26.7,32.5,22.7,NaN,43.3,7,7
4,여의나루,승차,우대권,16시,23.6,28.3,17.6,NaN,44.9,5,17
...,...,...,...,...,...,...,...,...,...,...,...
99995,마포,하차,외국인,13시,6.8,12.7,3.8,NaN,81.3,11,16
99996,김포공항,하차,일반,00시,27.0,30.4,24.4,1.3,67.3,8,21
99997,행당,승차,청소년,18시,6.8,14.5,0.5,NaN,41.0,2,23
99998,여의나루,하차,중고생,12시,19.6,26.9,12.9,NaN,44.5,5,14


In [8]:
# test 데이터 업무일자 월, 일로 나누기
date_list = list(test['업무일자'])
month = []
date = []

for day in date_list:
    month.append(int(day[5:7]))
    date.append(int(day[8:10]))
    
test['월'] = month
test['일'] = date

test = test.drop('업무일자',axis = 1)
test

,역명,승하차,사용자,시각,평균기온(℃),최고기온(℃),최저기온(℃),강수량(mm),평균습도(%rh),월,일
0,신길,하차,어린이,8시,1.4,3.0,0.2,4.6,80.8,2,19
1,종로3가,하차,우대권,17시,13.6,20.4,7.7,NaN,48.3,10,9
2,천호(풍납토성),승차,어린이,9시,-0.3,4.0,-3.2,0.0,45.8,1,17
3,오목교(목동운동장앞),승차,일반,17시,11.6,16.2,8.9,NaN,66.4,3,27
4,거여,하차,청소년,12시,1.7,7.1,-3.3,NaN,62.0,1,22
...,...,...,...,...,...,...,...,...,...,...,...
9995,마곡,승차,일반,17시,4.0,10.8,-1.6,NaN,56.1,2,22
9996,광화문(세종문화회관),하차,우대권,15시,-1.8,4.6,-6.8,NaN,47.4,2,17
9997,목동,하차,중고생,6시,23.9,28.9,20.6,NaN,64.4,7,1
9998,광화문(세종문화회관),하차,우대권,17시,18.9,23.7,14.8,NaN,60.5,6,1


In [9]:
set(train_x['시각'])

{'00시',
 '10시',
 '11시',
 '12시',
 '13시',
 '14시',
 '15시',
 '16시',
 '17시',
 '18시',
 '19시',
 '20시',
 '21시',
 '22시',
 '6시',
 '7시',
 '8시',
 '9시'}

In [10]:
#시각 전처리

t = train_x
time_list = []
for time in t['시각']:
    if time == "00시":
        time_list.append(0)
        continue
    
    idx = 0
    for i in range(1, 23):
        s = str(i)+"시"
        if s == time:
            time_list.append(i)
            idx = 1
            break
    
    if idx == 0:
        print(time)
        
train_x['시각'] = time_list


In [11]:
#Test 시각 전처리

t = test
time_list = []
for time in t['시각']:
    if time == "00시":
        time_list.append(0)
        continue
    
    idx = 0
    for i in range(1, 23):
        s = str(i)+"시"
        if s == time:
            time_list.append(i)
            idx = 1
            break
    
    if idx == 0:
        print(time)
        
test['시각'] = time_list

test

,역명,승하차,사용자,시각,평균기온(℃),최고기온(℃),최저기온(℃),강수량(mm),평균습도(%rh),월,일
0,신길,하차,어린이,8,1.4,3.0,0.2,4.6,80.8,2,19
1,종로3가,하차,우대권,17,13.6,20.4,7.7,NaN,48.3,10,9
2,천호(풍납토성),승차,어린이,9,-0.3,4.0,-3.2,0.0,45.8,1,17
3,오목교(목동운동장앞),승차,일반,17,11.6,16.2,8.9,NaN,66.4,3,27
4,거여,하차,청소년,12,1.7,7.1,-3.3,NaN,62.0,1,22
...,...,...,...,...,...,...,...,...,...,...,...
9995,마곡,승차,일반,17,4.0,10.8,-1.6,NaN,56.1,2,22
9996,광화문(세종문화회관),하차,우대권,15,-1.8,4.6,-6.8,NaN,47.4,2,17
9997,목동,하차,중고생,6,23.9,28.9,20.6,NaN,64.4,7,1
9998,광화문(세종문화회관),하차,우대권,17,18.9,23.7,14.8,NaN,60.5,6,1


In [12]:
len(set(train_x['사용자']))

7

In [13]:
from sklearn.preprocessing import LabelEncoder
encoder_station = LabelEncoder()
encoder_on_off = LabelEncoder()
encoder_user = LabelEncoder()

train_x['역명'] = encoder_station.fit_transform(train_x['역명'])
test['역명'] = encoder_station.transform(test['역명'])

train_x['승하차'] = encoder_on_off.fit_transform(train_x['승하차'])
test['승하차'] = encoder_on_off.transform(test['승하차'])

train_x['사용자'] = encoder_user.fit_transform(train_x['사용자'])
test['사용자'] = encoder_user.transform(test['사용자'])

In [14]:
#nan 값 데이터 처리
train_x['강수량(mm)'] = train_x['강수량(mm)'].fillna(value = 0)
test['강수량(mm)'] =  test['강수량(mm)'].fillna(value = 0)


In [15]:
train_x = train_x.drop(['월','일'], axis = 1)
test = test.drop(['월','일'], axis = 1)

In [16]:
display(train_x)
display(test)

,역명,승하차,사용자,시각,평균기온(℃),최고기온(℃),최저기온(℃),강수량(mm),평균습도(%rh)
0,21,0,1,8,7.0,12.7,3.1,0.0,51.1
1,40,1,5,21,0.5,6.3,-4.6,0.0,35.5
2,21,0,1,12,7.2,14.9,-0.4,0.0,31.8
3,25,1,4,8,26.7,32.5,22.7,0.0,43.3
4,34,0,3,16,23.6,28.3,17.6,0.0,44.9
...,...,...,...,...,...,...,...,...,...
99995,19,1,2,13,6.8,12.7,3.8,0.0,81.3
99996,11,1,4,0,27.0,30.4,24.4,1.3,67.3
99997,49,0,6,18,6.8,14.5,0.5,0.0,41.0
99998,34,1,5,12,19.6,26.9,12.9,0.0,44.5


,역명,승하차,사용자,시각,평균기온(℃),최고기온(℃),최저기온(℃),강수량(mm),평균습도(%rh)
0,29,1,1,8,1.4,3.0,0.2,4.6,80.8
1,45,1,3,17,13.6,20.4,7.7,0.0,48.3
2,46,0,1,9,-0.3,4.0,-3.2,0.0,45.8
3,39,0,4,17,11.6,16.2,8.9,0.0,66.4
4,3,1,6,12,1.7,7.1,-3.3,0.0,62.0
...,...,...,...,...,...,...,...,...,...
9995,16,0,4,17,4.0,10.8,-1.6,0.0,56.1
9996,7,1,3,15,-1.8,4.6,-6.8,0.0,47.4
9997,21,1,5,6,23.9,28.9,20.6,0.0,64.4
9998,7,1,3,17,18.9,23.7,14.8,0.0,60.5


In [17]:
from sklearn.preprocessing import StandardScaler
S = StandardScaler()
train_x_std = S.fit_transform(train_x)
test_std = S.transform(test)

In [18]:
x_train = torch.FloatTensor(train_x_std)
x_test = torch.FloatTensor(test_std)
y_train = torch.FloatTensor(train_y.values)

y_train = y_train.view([-1, 1])

# 4) 학습 모델 설정
* 파라미터 설정
* layer 8개 이용

In [19]:
# 학습 파라미터 설정
learning_rate = 0.001
training_epochs = 120 #500 ->100
batch_size = 240
drop_prob = 0.01

In [20]:
# learning_rate = 0.001
# training_epochs = 5
# batch_size = 100
# method = "resnet"

In [21]:
from torch.utils.data import  TensorDataset, DataLoader
dataSet = TensorDataset(x_train, y_train)
data_loader = torch.utils.data.DataLoader(dataset=dataSet,
                                          batch_size=batch_size,
                                          shuffle=True,
                                          drop_last=True)
data_loader

In [22]:
linear1 = torch.nn.Linear(9,256,bias=True)
linear2 = torch.nn.Linear(256,512,bias=True)
linear5 = torch.nn.Linear(512,512,bias=True)
linear6 = torch.nn.Linear(512,512,bias=True)
linear7 = torch.nn.Linear(512,512,bias=True)
linear3 = torch.nn.Linear(512,512,bias=True)
linear8 = torch.nn.Linear(512,128,bias=True)
linear4 = torch.nn.Linear(128,1,bias=True)
# linear5 = torch.nn.Linear(1000,1000,bias=True)
# linear6 = torch.nn.Linear(1000,1000,bias=True)
# linear7 = torch.nn.Linear(1000,1,bias=True)
relu = torch.nn.ReLU()
dropout = torch.nn.Dropout(p=drop_prob)

# Random Init => Xavier Init
torch.nn.init.xavier_uniform_(linear1.weight)
torch.nn.init.xavier_uniform_(linear2.weight)
torch.nn.init.xavier_uniform_(linear5.weight)
torch.nn.init.xavier_uniform_(linear6.weight)
torch.nn.init.xavier_uniform_(linear7.weight)

torch.nn.init.xavier_uniform_(linear3.weight)
torch.nn.init.xavier_uniform_(linear4.weight)


Parameter containing:
tensor([[ 0.0123,  0.1483, -0.1098,  0.2136,  0.1511, -0.0480, -0.1504,  0.2133,
          0.0782,  0.0873,  0.1566, -0.1254, -0.2059,  0.0957, -0.0339,  0.2087,
         -0.0904, -0.1656,  0.0400,  0.0606,  0.0483,  0.0537, -0.0986,  0.0812,
          0.0399, -0.0743,  0.1118,  0.1568, -0.1467, -0.0075,  0.0923,  0.0260,
         -0.1363, -0.1937, -0.0381,  0.0564,  0.1440, -0.1104,  0.0404,  0.0552,
         -0.2054, -0.0680, -0.1557,  0.0540,  0.1314, -0.0184,  0.0539,  0.1731,
         -0.0355,  0.0106, -0.1272,  0.1347, -0.1326,  0.0763,  0.0796,  0.1806,
         -0.1136,  0.0503,  0.0091,  0.0276,  0.0145,  0.0441, -0.0062, -0.0839,
          0.1900, -0.1321, -0.0450, -0.1125,  0.2127,  0.2047,  0.0765,  0.1464,
         -0.2114,  0.1098, -0.0958, -0.0320,  0.0198, -0.0035, -0.1853, -0.1581,
          0.1010,  0.1821, -0.1939,  0.0600, -0.0910, -0.0360,  0.0426,  0.1596,
         -0.1472, -0.2045,  0.1460, -0.0508, -0.1251,  0.1889,  0.1263, -0.1050,
      

In [23]:
# ======================================
# relu는 맨 마지막 레이어에서 빼는 것이 좋다.
# ======================================
model = torch.nn.Sequential(linear1,relu,dropout,
                            linear2,relu,dropout,
                            linear5,relu,dropout,
                            linear6,relu,dropout,
                            linear7,relu,dropout,
                            linear3,relu,dropout,
                            linear8,relu,dropout,
                            linear4).to(device)


# 손실함수와 최적화 함수
loss = torch.nn.MSELoss().to(device) # softmax 내부적으로 계산
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate) 

# 5) Train data로 모델 학습

In [24]:
total_batch = len(data_loader)
for epoch in range(training_epochs):
    avg_cost = 0

    for X, Y in data_loader:

        X = X.to(device)
        Y = Y.to(device)

        # 그래디언트 초기화
        optimizer.zero_grad()
        # Forward 계산
        hypothesis = model(X)
        
        # Error 계산
        cost = loss(hypothesis, Y)
        # Backparopagation
        cost.backward()
        # 가중치 갱신
        optimizer.step()

        # 평균 Error 계산
        avg_cost += cost / total_batch
        
    if epoch % 10 == 0:
        print('Epoch:', '%04d' % (epoch + 1), 'cost =', '{:.9f}'.format(avg_cost))

print('Learning finished')

Epoch: 0001 cost = 106759.843750000
Epoch: 0011 cost = 83736.109375000
Epoch: 0021 cost = 82586.601562500
Epoch: 0031 cost = 77577.125000000
Epoch: 0041 cost = 76405.234375000
Epoch: 0051 cost = 69343.890625000
Epoch: 0061 cost = 69698.601562500
Epoch: 0071 cost = 67020.937500000
Epoch: 0081 cost = 63730.207031250
Epoch: 0091 cost = 59927.269531250
Epoch: 0101 cost = 57117.777343750
Epoch: 0111 cost = 60239.816406250
Learning finished


# 6) Test data 예측 결과

In [25]:
# Test the model using test sets
with torch.no_grad():
    model.eval()  # 주의사항 (dropout=False)
    X_test = x_test.to(device)

    prediction = model(X_test)
prediction

tensor([[  1.3399],
        [294.8591],
        [  1.9550],
        ...,
        [  1.9175],
        [163.5401],
        [108.9297]], device='cuda:0')

In [26]:
sample

,index,predict
0,1,0
1,2,0
2,3,0
3,4,0
4,5,0
...,...,...
9995,9996,0
9996,9997,0
9997,9998,0
9998,9999,0


In [27]:
sample['predict'] = prediction.cpu().detach()


In [28]:
# sample = sample.drop('탑승인원', axis = 1)
sample.to_csv('submit.csv', index = None)

In [29]:
sample

,index,predict
0,1,1.339875
1,2,294.859070
2,3,1.955040
3,4,633.662476
4,5,36.058197
...,...,...
9995,9996,508.661896
9996,9997,157.712936
9997,9998,1.917549
9998,9999,163.540054
